<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# `GiRaFFEfood`: Initial data for `GiRaFFE` Aligned Rotator

## Author: Zach Etienne & Patrick Nelson
### Formatting improvements courtesy Brandon Clark

[comment]: <> (Abstract: TODO)

**Notebook Status:** <font color='green'><b> Validated </b></font>

**Validation Notes:** This tutorial notebook has been confirmed to be self-consistent with its corresponding NRPy+ module, as documented [below](#code_validation). The initial data has validated against the original `GiRaFFE`, as documented [here](Tutorial-Start_to_Finish_UnitTest-GiRaFFEfood_NRPy.ipynb).

### NRPy+ Source Code for this module: [GiRaFFEfood_NRPy_Aligned_Rotator.py](../../edit/in_progress/GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_Aligned_Rotator.py)

## Introduction: 

This module provides another initial data option for `GiRaFFE`. This is a flat-spacetime test with initial data $$A_{\phi} = \frac{\mu \varpi^2}{r^3},$$ where  $\mu = B_p R^3_{\rm NS} / 2$, $R_{\rm NS}$ is the neutron star radius, and $\varpi = \sqrt{x^2+y^2}$ is the cylindrical radius. We let $A_r = A_\theta = 0$.

Additionally, the drift velocity $v^i = \Omega \textbf{e}_z \times \textbf{r} = [ijk] \Omega \textbf{e}^j_z x^k$, where $[ijk]$ is the Levi-Civita permutation symbol, $\Omega$ is the angular velocity of the "neutron star", and $\textbf{e}^i_z = (0,0,1)$. See the [Tutorial-GiRaFFEfood_NRPy](Tutorial-GiRaFFEfood_NRPy.ipynb) tutorial notebook for details on how this is used.

<a id='toc'></a>

# Table of Contents:
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#initializenrpy): Import core NRPy+ modules and set NRPy+ parameters
1. [Step 2](#set_a_i): Set the vector $A_i$
1. [Step 3](#set_vi): Calculate $v^i$
1. [Step 4](#code_validation1): Code Validation against `GiRaFFEfood_NRPy.GiRaFFEfood_NRPy` NRPy+ Module
1. [Step 5](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='initializenrpy'></a>

# Step 1: Import core NRPy+ modules and set NRPy+ parameters \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

Here, we will import the NRPy+ core modules and set the reference metric to Cartesian, set commonly used NRPy+ parameters, and set C parameters that will be set from outside the code eventually generated from these expressions. We will also set the parameters we need for this module.

In [1]:
# Step 0: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import os,sys
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

# Step 0.a: Import the NRPy+ core modules and set the reference metric to Cartesian
import sympy as sp               # SymPy: The Python computer algebra package upon which NRPy+ depends
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import GiRaFFEfood_NRPy.GiRaFFEfood_NRPy_Common_Functions as gfcf # Some useful functions for GiRaFFE initial data.

import reference_metric as rfm
par.set_parval_from_str("reference_metric::CoordSystem","Cartesian")
rfm.reference_metric()

# Step 1a: Set commonly used parameters.
thismodule = "GiRaFFEfood_NRPy_Aligned_Rotator"

# The angular velocity of the "neutron star"
Omega_aligned_rotator = par.Cparameters("REAL",thismodule,"Omega_aligned_rotator",1e3)
B_p_aligned_rotator,R_NS_aligned_rotator = par.Cparameters("REAL",thismodule,
                                                           # B_p_aligned_rotator = the intensity of the magnetic field and
                                                           # R_NS_aligned_rotator= "Neutron star" radius
                                                           ["B_p_aligned_rotator","R_NS_aligned_rotator"],
                                                           [1e-5, 1.0])

<a id='set_a_i'></a>

# Step 2: Set the vector $A_i$ \[Back to [top](#toc)\]
$$\label{set_a_i}$$


We will first build the fundamental vector $A_i$ in spherical coordinates (see [Table 3](https://arxiv.org/pdf/1704.00599.pdf)). Note that we use [reference_metric.py](../edit/reference_metric.py) to set $r$ and $\varpi$ in terms of Cartesian coordinates; this will save us a step later when we convert to Cartesian coordinates. So, we set 
\begin{align}
A_{\phi} &= \frac{\mu \varpi^2}{r^3}, \\
\end{align}
with $\mu = B_p R^3_{\rm NS} / 2$, $R_{\rm NS}$ is the neutron star radius, and $\varpi = \sqrt{x^2+y^2} = r \sin(\theta)$


In [2]:
# Step 2: Set the vectors A and E in Spherical coordinates
def Ar_AR(r,theta,phi, **params):
    return sp.sympify(0)

def Ath_AR(r,theta,phi, **params):
    return sp.sympify(0)

def Aph_AR(r,theta,phi, **params):
    # \mu \varpi / r^3
    # \varpi = sqrt(x^2+y^2) = r \sin(\theta)
    varpi = r * sp.sin(theta)
    mu = B_p_aligned_rotator * R_NS_aligned_rotator**3 / 2
    return mu * varpi**2 / (r**3)

<a id='set_vi'></a>

# Step 3: Calculate $v^i$ \[Back to [top](#toc)\]
$$\label{set_vi}$$

Here, we will calculate the drift velocity $v^i = \Omega \textbf{e}_z \times \textbf{r} = [ijk] \Omega \textbf{e}^j_z x^k$, where $[ijk]$ is the Levi-Civita permutation symbol, $\Omega$ is the angular velocity of the "neutron star", and $\textbf{e}^i_z = (0,0,1)$. Conveniently, in flat space, the drift velocity reduces to the Valencia velocity because $\alpha = 1$ and $\beta^i = 0$.

However, we need to set the velocity to zero outside the neutron star; this is equivalent to only setting the non-zero velocity inside the star, $r \leq R_{\rm NS}$

In [3]:
import Min_Max_and_Piecewise_Expressions as noif
#Step 3: Compute v^i
def ValenciavU_func_AR(**params):
    LeviCivitaSymbolDDD = ixp.LeviCivitaSymbol_dim3_rank3()

    unit_zU = ixp.zerorank1()
    unit_zU[2] = sp.sympify(1)

    r = rfm.xxSph[0]

    ValenciavU = ixp.zerorank1()
    for i in range(3):
        for j in range(3):
            for k in range(3):
                ValenciavU[i] += noif.coord_leq_bound(r,R_NS_aligned_rotator)*LeviCivitaSymbolDDD[i][j][k] * Omega_aligned_rotator * unit_zU[j] * rfm.xx[k]

    return ValenciavU

<a id='code_validation'></a>

# Step 4: Code Validation against `GiRaFFEfood_NRPy.GiRaFFEfood_NRPy` NRPy+ Module  \[Back to [top](#toc)\]
$$\label{code_validation}$$

Here, as a code validation check, we verify agreement in the SymPy expressions for the `GiRaFFE` Aligned Rotator initial data equations we intend to use between

1. this tutorial and 
2. the NRPy+ [GiRaFFEfood_NRPy.GiRaFFEfood_NRPy_Aligned_Rotator](../edit/GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_Aligned_Rotator.py) module.

In [4]:
import GiRaFFEfood_NRPy.GiRaFFEfood_NRPy as gf

A_arD = gfcf.Axyz_func_spherical(Ar_AR,Ath_AR,Aph_AR,stagger_enable = True,)
Valenciav_arD = ValenciavU_func_AR()
gf.GiRaFFEfood_NRPy_generate_initial_data(ID_type = "AlignedRotator", stagger_enable = True)

def consistency_check(quantity1,quantity2,string):
    if quantity1-quantity2==0:
        print(string+" is in agreement!")
    else:
        print(string+" does not agree!")
        sys.exit(1)

print("Consistency check between GiRaFFEfood_NRPy tutorial and NRPy+ module:")

for i in range(3):
    consistency_check(Valenciav_arD[i],gf.ValenciavU[i],"ValenciavU"+str(i))
    consistency_check(A_arD[i],gf.AD[i],"AD"+str(i))

Consistency check between GiRaFFEfood_NRPy tutorial and NRPy+ module:
ValenciavU0 is in agreement!
AD0 is in agreement!
ValenciavU1 is in agreement!
AD1 is in agreement!
ValenciavU2 is in agreement!
AD2 is in agreement!


<a id='latex_pdf_output'></a>

# Step 5: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-GiRaFFEfood_NRPy_Aligned_Rotator.pdf](Tutorial-GiRaFFEfood_NRPy_Aligned_Rotator.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [5]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-GiRaFFEfood_NRPy-Aligned_Rotator",location_of_template_file=os.path.join(".."))

Created Tutorial-GiRaFFEfood_NRPy-Aligned_Rotator.tex, and compiled LaTeX
    file to PDF file Tutorial-GiRaFFEfood_NRPy-Aligned_Rotator.pdf
